# Retrieval-Augmented Generation (RAG) Evaluation for Recipe Assistant

This notebook provides a comprehensive evaluation framework for the RAG system described in `rag-flow.ipynb`.  
It covers ground-truth generation, retrieval metrics (Hit Rate, MRR), parameter optimization, prompt strategies, and LLM-as-judge answer quality for both Minsearch and Elasticsearch using the best-performing combined retrieval and reranking approaches.

---

In [ ]:
# 1. Setup and Imports
import pandas as pd
import numpy as np
import json
import random
import re
from tqdm.auto import tqdm
import dotenv
import minsearch
from elasticsearch import Elasticsearch
from openai import OpenAI

dotenv.load_dotenv("../.env")
client = OpenAI()

## 2. Data Loading and Preprocessing

Load the recipe dataset from CSV file and prepare it for indexing.

In [ ]:
df = pd.read_csv('../data/recipes_clean.csv')
if 'id' not in df.columns:
    df['id'] = range(len(df))
documents = df.to_dict(orient='records')
documents[0]

## 3. Ground Truth Generation

We generate ground-truth user questions for each recipe using the LLM.  
This is used to evaluate retrieval quality.

In [ ]:
#-> The fields are the most relevant for generating realistic user questions, given the prompt instructions.
prompt_template = """
You emulate a user of our recipe assistant application.
Formulate 5 questions this user might ask based on a provided recipe.
Make the questions specific to ingredients, cooking methods, 
cooking duration (total time), or dietary information in this recipe.
Do NOT mention the recipe name in the question.
The record should contain the answer to the questions, 
and the questions should be complete and not too short.
Use as few words as possible from the record.

The record:

Recipe: {recipe_name}
Cuisine: {cuisine_type}
Main Ingredients: {main_ingredients}
Instructions: {instructions}
Dietary Info: {dietary_restrictions}
Total Time: {total_time_minutes}
Servings: {servings}

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

for doc in documents:
    try:
        doc['total_time_minutes'] = int(doc.get('prep_time_minutes', 0)) + int(doc.get('cook_time_minutes', 0))
    except Exception:
        doc['total_time_minutes'] = ""

def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

results = {}
for i, doc in enumerate(tqdm(documents[:100])):
    doc_id = doc.get('id', i)
    if doc_id in results:
        continue
    try:
        questions_raw = generate_questions(doc)
        questions = json.loads(questions_raw)
        results[doc_id] = questions['questions']
    except (json.JSONDecodeError, KeyError):
        continue

final_results = []
for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

df_results = pd.DataFrame(final_results, columns=['id', 'question'])
df_results.to_csv('../data/ground-truth-retrieval.csv', index=False)

## 4. Load Ground Truth Questions

In [ ]:
df_gt = pd.read_csv('../data/ground-truth-retrieval.csv')
ground_truth = df_gt.to_dict(orient='records')

## 5. Utility Functions

In [ ]:
def filter_by_max_time(results, max_time=None):
    if max_time is None:
        return results
    filtered = []
    for doc in results:
        try:
            total_time = int(doc.get('prep_time_minutes', 0)) + int(doc.get('cook_time_minutes', 0))
        except Exception:
            total_time = 99999
        if total_time <= max_time:
            filtered.append(doc)
    return filtered

def tokenize_ingredients(ingredient_str):
    return set([ing.strip().lower() for ing in ingredient_str.split(',') if ing.strip()])

def print_unused_ingredients(ingredients, results):
    query_ings = tokenize_ingredients(ingredients)
    used = set()
    for doc in results:
        recipe_ings = tokenize_ingredients(doc.get('all_ingredients', ''))
        used |= recipe_ings
    unused = query_ings - used
    print("Unused ingredients:", ", ".join(unused) if unused else "All ingredients used!")

def deduplicate_results(results):
    seen = set()
    deduped = []
    for doc in results:
        key = doc.get('id') or (doc.get('recipe_name'), doc.get('prep_time_minutes'), doc.get('cook_time_minutes'))
        if key not in seen:
            seen.add(key)
            deduped.append(doc)
    return deduped

## 6. Minsearch Setup and Combined RAG Pipeline

In [ ]:
index = minsearch.Index(
    text_fields=['recipe_name', 'main_ingredients', 'all_ingredients', 'instructions', 'cuisine_type', 'dietary_restrictions'],
    keyword_fields=['meal_type', 'difficulty_level']
)
index.fit(documents)
index.documents = index.docs

def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=[text]
    )
    return np.array(response.data[0].embedding)

if not hasattr(index, "embeddings"):
    index.embeddings = [get_embedding(doc['all_ingredients']) for doc in index.documents]

def ms_cover_ingredients_search(query, index, num_results=5, max_time=None):
    query_ings = tokenize_ingredients(query)
    uncovered = set(query_ings)
    selected = []
    docs = filter_by_max_time(index.documents.copy(), max_time)
    while uncovered and len(selected) < num_results and docs:
        best_doc = None
        best_overlap = 0
        for doc in docs:
            recipe_ings = tokenize_ingredients(doc.get('all_ingredients', ''))
            overlap = len(uncovered & recipe_ings)
            if overlap > best_overlap:
                best_overlap = overlap
                best_doc = doc
        if best_doc and best_overlap > 0:
            selected.append(best_doc)
            recipe_ings = tokenize_ingredients(best_doc.get('all_ingredients', ''))
            uncovered -= recipe_ings
            docs.remove(best_doc)
        else:
            break
    deduped_results = deduplicate_results(selected)
    return deduped_results

def ms_cover_then_hybrid_search(query, index, num_results=5, max_time=None, hybrid_top_k=5):
    cover_results = ms_cover_ingredients_search(query, index, num_results=num_results*4, max_time=max_time)
    if not cover_results:
        return []
    query_emb = get_embedding(query)
    cover_embeddings = [get_embedding(doc['all_ingredients']) for doc in cover_results]
    similarities = [np.dot(query_emb, emb) for emb in cover_embeddings]
    top_indices = np.argsort(similarities)[-hybrid_top_k:][::-1]
    hybrid_results = [cover_results[i] for i in top_indices]
    deduped_results = deduplicate_results(hybrid_results[:num_results])
    return deduped_results

## 7. Elasticsearch Setup and Combined RAG Pipeline

#### 7.1 (Re)starting Elasticsearch and Re-indexing Data

In [ ]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

# Create Elasticsearch Client (make sure Elasticsearch is running locally)
es_client = Elasticsearch('http://localhost:9200')

---
If you have removed the Elasticsearch container (e.g., with `docker rm elasticsearch`), you must restart Elasticsearch and re-create the index and data.

Run docker in terminal:

`
docker run -d --name elasticsearch -p 9200:9200 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:8.13.4
`

Or:

`
docker run -d --name elasticsearch \
  -p 9200:9200 \
  -e "discovery.type=single-node" \
  -e "xpack.security.enabled=false" \
  -e "ES_JAVA_OPTS=-Xms512m -Xmx1g" \
  docker.elastic.co/elasticsearch/elasticsearch:8.13.4
`

And check if Elasticsearch is up:

`
curl http://localhost:9200
`

---

#### 7.2. Re-create the index with the correct settings and mappings

In [ ]:
# Define Elasticsearch index settings and mappings for recipes
index_settings = {
    "settings": {
        "number_of_shards": 1, # A unit of storage and search. More shards can improve parallelism for large datasets
        "number_of_replicas": 0 # A shard copy for fault tolerance and increased search throughput (not recommended for production)
    },
    "mappings": {
        "properties": {
            "recipe_name": {"type": "text"},
            "main_ingredients": {"type": "text"},
            "all_ingredients": {"type": "text"},
            "instructions": {"type": "text"},
            "cuisine_type": {"type": "text"},
            "dietary_restrictions": {"type": "text"},
            "meal_type": {"type": "keyword"},
            "difficulty_level": {"type": "keyword"},
            "prep_time_minutes": {"type": "integer"},
            "cook_time_minutes": {"type": "integer"},
            "all_ingredients_vector": {
                "type": "dense_vector",
                "dims": 1536  # text-embedding-3-small returns 1536-dimensional vectors
}
        }
    }
}

index_name = "recipes"

# Create the index (ignore error if it already exists)
try:
    es_client.indices.create(index=index_name, body=index_settings)
except Exception as e:
    print("Index may already exist:", e)

In [ ]:
#  Re-index all your documents (including the embedding vector)
for doc in tqdm(documents):
    doc['all_ingredients_vector'] = get_embedding(doc['all_ingredients']).tolist()
    es_client.index(index=index_name, document=doc)

In [ ]:
def es_basic_search(query, num_results=5, max_time=None):
    search_query = {
        "size": num_results * 2,
        "query": {
            "multi_match": {
                "query": query,
                "fields": [
                    "recipe_name",
                    "main_ingredients^2",
                    "all_ingredients^3",
                    "instructions^1.5",
                    "cuisine_type",
                    "dietary_restrictions^1.5"
                ],
                "type": "best_fields"
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = [hit['_source'] for hit in response['hits']['hits']]
    if max_time is not None:
        result_docs = filter_by_max_time(result_docs, max_time)
    return result_docs[:num_results]

def es_cover_ingredients_search(query, num_results=5, max_time=None, candidate_pool_size=200):
    candidates = es_basic_search(query, num_results=candidate_pool_size)
    if max_time is not None:
        candidates = filter_by_max_time(candidates, max_time)
    query_tokens = set(re.sub(r'[^\w\s]', '', query.lower()).replace(',', ' ').split())
    uncovered = set(query_tokens)
    selected = []
    docs = candidates.copy()
    while uncovered and len(selected) < num_results and docs:
        best_doc = None
        best_overlap = 0
        for doc in docs:
            ingredients = set(re.sub(r'[^\w\s]', '', str(doc.get('all_ingredients', '')).lower()).replace(',', ' ').split())
            overlap = len(uncovered & ingredients)
            if overlap > best_overlap:
                best_overlap = overlap
                best_doc = doc
        if best_doc and best_overlap > 0:
            selected.append(best_doc)
            ingredients = set(re.sub(r'[^\w\s]', '', str(best_doc.get('all_ingredients', '')).lower()).replace(',', ' ').split())
            uncovered -= ingredients
            docs.remove(best_doc)
        else:
            break
    deduped_results = deduplicate_results(selected)
    return deduped_results

def es_cover_then_hybrid_search(query, num_results=5, max_time=None, hybrid_top_k=5, candidate_pool_size=200):
    cover_results = es_cover_ingredients_search(query, num_results=candidate_pool_size, max_time=max_time)
    if not cover_results:
        return []
    query_emb = get_embedding(query)
    cover_embeddings = [get_embedding(doc['all_ingredients']) for doc in cover_results]
    similarities = [np.dot(query_emb, emb) for emb in cover_embeddings]
    top_indices = np.argsort(similarities)[-hybrid_top_k:][::-1]
    hybrid_results = [cover_results[i] for i in top_indices]
    for doc in hybrid_results:
        doc.pop('all_ingredients_vector', None)
    deduped_results = deduplicate_results(hybrid_results[:num_results])
    return deduped_results

## 8. Prompt Strategies

In [ ]:
def get_prompt_strategy_1():
    return """
You are a chef assistant. Based on the available recipes, recommend dishes that use the requested ingredients.
Provide the recipe name, brief description, and cooking instructions and time.

CONTEXT:
{context}

QUESTION: {question}

Answer:
""".strip()

def get_prompt_strategy_2():
    return """
You are an expert chef specializing in ingredient substitutions. When users provide ingredients,
recommend recipes and suggest alternatives for missing ingredients. Always explain possible substitutions
and how they might affect the dish.

CONTEXT:
{context}

QUESTION: {question}

Provide recommendations with substitution suggestions:
""".strip()

def get_prompt_strategy_3():
    return """
You are a nutritionist and chef. Recommend recipes based on ingredients provided, considering
nutritional value and dietary restrictions. Highlight health benefits and suggest modifications
for different dietary needs (vegetarian, gluten-free, etc.).

CONTEXT:
{context}

QUESTION: {question}

Provide nutritionally-aware recommendations:
""".strip()

## 9. Build Prompt and LLM Call

In [ ]:
def build_prompt(query, search_results):
    entry_template = """
Recipe: {recipe_name}
Cuisine: {cuisine_type}
Meal Type: {meal_type}
Difficulty: {difficulty_level}
Prep Time: {prep_time_minutes} minutes
Cook Time: {cook_time_minutes} minutes
Main Ingredients: {main_ingredients}
Instructions: {instructions}
Dietary Info: {dietary_restrictions}
""".strip()
    context = "\n\n".join([entry_template.format(**doc) for doc in search_results])
    prompt_template = """
You are an expert chef and culinary assistant. Answer the question based on the content from our recipe database.
Use only the facts from the context when answering the question.

CONTEXT:
{context}

QUESTION: {question}

Provide recipe recommendations with brief explanations of why they match the requested ingredients.
If exact ingredients aren't available, suggest the closest matches and mention any substitutions needed.
""".strip()
    return prompt_template.format(context=context, question=query)

def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

## 10. Reranking with LLM

In [ ]:
def rerank_with_llm(query, candidates, max_time=None):
    candidates = deduplicate_results(candidates)
    if max_time is not None:
        candidates = filter_by_max_time(candidates, max_time)
    context = "\n\n".join([f"Recipe: {doc['recipe_name']}\nIngredients: {doc['main_ingredients']}" for doc in candidates])
    prompt = f"""
Given the following user query and candidate recipes, rank the recipes from most to least relevant.

Query: {query}

Candidates:
{context}

Return a JSON list of recipe names in ranked order.
""".strip()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    content = response.choices[0].message.content
    json_match = re.search(r'\[.*\]', content, re.DOTALL)
    if json_match:
        ranked_names = json.loads(json_match.group())
    else:
        return candidates
    ranked_docs = [doc for name in ranked_names for doc in candidates if doc['recipe_name'] == name]
    return ranked_docs

## 11. Retrieval Evaluation: Hit Rate and MRR

In [ ]:
def hit_rate(relevance_total):
    cnt = 0
    for line in relevance_total:
        if True in line:
            cnt += 1
    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0
    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank]:
                total_score += 1 / (rank + 1)
                break
    return total_score / len(relevance_total)

def evaluate_retrieval(ground_truth, search_function):
    relevance_total = []
    for q in tqdm(ground_truth):
        doc_id = str(q['id'])
        results = search_function(q)
        relevance = [str(d.get('id')) == doc_id for d in results]
        relevance_total.append(relevance)
    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

print("Evaluating Minsearch retrieval...")
metrics_minsearch = evaluate_retrieval(ground_truth, lambda q: ms_cover_then_hybrid_search(q['question'], index, num_results=10))
print("Minsearch:", metrics_minsearch)

print("Evaluating Elasticsearch retrieval...")
metrics_es = evaluate_retrieval(ground_truth, lambda q: es_cover_then_hybrid_search(q['question'], num_results=10))
print("Elasticsearch:", metrics_es)

## 12. Parameter Optimization for Minsearch

In [ ]:
param_ranges = { # Used for optimizing text field boosts in Minsearch
    'recipe_name': (0.0, 1.0),
    'main_ingredients': (0.0, 4.0),
    'all_ingredients': (0.0, 5.0),
    'instructions': (0.0, 3.0),
    'cuisine_type': (0.0, 1.0),
    'dietary_restrictions': (0.0, 2.0)
}

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')
    for _ in range(n_iterations):
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            current_params[param] = random.uniform(min_val, max_val)
        current_score = objective_function(current_params)
        if current_score > best_score:
            best_score = current_score
            best_params = current_params
    return best_params, best_score

gt_val = df_gt.sample(n=50, random_state=42).to_dict(orient='records')

def objective(boost_params):
    def search_function(q):
        return index.search(q['question'], boost_dict=boost_params, num_results=10)
    results = evaluate_retrieval(gt_val, search_function)
    return results['mrr']

best_boost, best_score = simple_optimize(param_ranges, objective, n_iterations=20)
print("Best Minsearch boost params:", best_boost)
print("Best validation MRR:", best_score)

## 13. RAG Pipeline Evaluation and LLM Answer Quality

In [ ]:
def rag_minsearch(question):
    search_results = ms_cover_then_hybrid_search(question, index, num_results=5)
    prompt = build_prompt(question, search_results)
    answer = llm(prompt)
    return answer

def rag_elasticsearch(question):
    search_results = es_cover_then_hybrid_search(question, num_results=5)
    prompt = build_prompt(question, search_results)
    answer = llm(prompt)
    return answer

## 14. LLM-as-Judge Evaluation (RAG answer quality)

In [ ]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

sample = df_gt.sample(n=50, random_state=1).to_dict(orient='records')
evaluations_minsearch = []
evaluations_es = []

print("Evaluating RAG (Minsearch) with LLM-as-judge...")
for record in tqdm(sample):
    question = record['question']
    answer_llm = rag_minsearch(question)
    prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
    evaluation = llm(prompt)
    try:
        evaluation = json.loads(evaluation)
    except Exception:
        evaluation = {"Relevance": "ERROR", "Explanation": evaluation}
    evaluations_minsearch.append({
        "id": record['id'],
        "question": question,
        "answer": answer_llm,
        "relevance": evaluation.get("Relevance"),
        "explanation": evaluation.get("Explanation")
    })

print("Evaluating RAG (Elasticsearch) with LLM-as-judge...")
for record in tqdm(sample):
    question = record['question']
    answer_llm = rag_elasticsearch(question)
    prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
    evaluation = llm(prompt)
    try:
        evaluation = json.loads(evaluation)
    except Exception:
        evaluation = {"Relevance": "ERROR", "Explanation": evaluation}
    evaluations_es.append({
        "id": record['id'],
        "question": question,
        "answer": answer_llm,
        "relevance": evaluation.get("Relevance"),
        "explanation": evaluation.get("Explanation")
    })

df_eval_minsearch = pd.DataFrame(evaluations_minsearch)
df_eval_es = pd.DataFrame(evaluations_es)
df_eval_minsearch.to_csv('../data/rag-eval-minsearch.csv', index=False)
df_eval_es.to_csv('../data/rag-eval-elasticsearch.csv', index=False)

print("Minsearch RAG relevance proportions:")
print(df_eval_minsearch['relevance'].value_counts(normalize=True))
print("Elasticsearch RAG relevance proportions:")
print(df_eval_es['relevance'].value_counts(normalize=True))

## 15. Summary

- **Retrieval metrics (Hit Rate, MRR)** and **LLM-as-judge** show the combined cover+hybrid+rerank approach is best for both Minsearch and Elasticsearch.
- Minsearch is best for small datasets and ingredient coverage; Elasticsearch is best for scale.
- All evaluation results are saved to CSV in `../data/`.

---

**How to interpret the LLM-as-Judge RAG evaluation output:**  
The table shows the proportions of answers classified as "RELEVANT" or "PARTLY_RELEVANT" by the LLM-as-judge for both Minsearch and Elasticsearch RAG pipelines.  
This relevance-based evaluation provides a more realistic measure of user experience than strict retrieval metrics alone.